In [7]:
from illuminating.data_preparation.geodata_processing import download_gdf

In [8]:
USA_gdf = download_gdf("USA",1)

In [9]:
USA_gdf.head()

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,USA.1_1,USA,UnitedStates,Alabama,AL|Ala.,NA,State,State,NA,US.AL,US-AL,"MULTIPOLYGON (((-88.12060 30.23320, -88.12330 ..."
1,USA.2_1,USA,UnitedStates,Alaska,AK|Alaska,NA,State,State,NA,US.AK,US-AK,"MULTIPOLYGON (((-179.13150 51.20970, -179.1328..."
2,USA.3_1,USA,UnitedStates,Arizona,AZ|Ariz.,NA,State,State,NA,US.AZ,US-AZ,"MULTIPOLYGON (((-110.75620 31.33290, -111.0750..."
3,USA.4_1,USA,UnitedStates,Arkansas,AR|Ark.,NA,State,State,NA,US.AR,US-AR,"MULTIPOLYGON (((-91.90710 33.00800, -92.86190 ..."
4,USA.5_1,USA,UnitedStates,California,CA|Calif.,NA,State,State,NA,US.CA,US-CA,"MULTIPOLYGON (((-117.23300 32.77420, -117.2336..."


In [10]:
from re import finditer
def camel_case_split(identifier):
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0) for m in matches])

In [11]:
string = "NewYork"

In [5]:
states = USA_gdf.NAME_1.apply(camel_case_split)
states

0                 Alabama
1                  Alaska
2                 Arizona
3                Arkansas
4              California
5                Colorado
6             Connecticut
7                Delaware
8     Districtof Columbia
9                 Florida
10                Georgia
11                 Hawaii
12                  Idaho
13               Illinois
14                Indiana
15                   Iowa
16                 Kansas
17               Kentucky
18              Louisiana
19                  Maine
20               Maryland
21          Massachusetts
22               Michigan
23              Minnesota
24            Mississippi
25               Missouri
26                Montana
27               Nebraska
28                 Nevada
29          New Hampshire
30             New Jersey
31             New Mexico
32               New York
33         North Carolina
34           North Dakota
35                   Ohio
36               Oklahoma
37                 Oregon
38          

In [6]:
states = states.replace("Districtof Columbia", "District of Columbia")

In [53]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = f"""
[out:json];
(
  way["power"]({min_lat},{min_lon},{max_lat},{max_lon});
  node["power"]({min_lat},{min_lon},{max_lat},{max_lon});
  relation["power"]({min_lat},{min_lon},{max_lat},{max_lon});
);
out body;
>;
out skel qt;
"""

# Send the request to the Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})

# Check if the request was successful
if response.status_code == 200:
    osm_data = response.json()
    
    # Convert the data to GeoJSON
    features = []
    nodes = {node['id']: (node['lat'], node['lon']) for node in osm_data['elements'] if node['type'] == 'node'}
    
    for element in osm_data['elements']:
        if element['type'] == 'way':
            coordinates = [(nodes[node_id][1], nodes[node_id][0]) for node_id in element['nodes']]
            feature = geojson.Feature(
                geometry=geojson.LineString(coordinates),
                properties={
                    "id": element['id'],
                    "power": element.get('tags', {}).get('power', ''),
                    "tags": element.get('tags', {})
                }
            )
            features.append(feature)
        elif element['type'] == 'node':
            feature = geojson.Feature(
                geometry=geojson.Point((nodes[element['id']][1], nodes[element['id']][0])),
                properties={
                    "id": element['id'],
                    "power": element.get('tags', {}).get('power', ''),
                    "tags": element.get('tags', {})
                }
            )
            features.append(feature)
        elif element['type'] == 'relation':
            # Handle relations similarly to ways and nodes, depending on member types
            for member in element['members']:
                if member['type'] == 'way' and member['ref'] in nodes:
                    coordinates = [(nodes[node_id][1], nodes[node_id][0]) for node_id in element['nodes']]
                    feature = geojson.Feature(
                        geometry=geojson.LineString(coordinates),
                        properties={
                            "id": element['id'],
                            "power": element.get('tags', {}).get('power', ''),
                            "tags": element.get('tags', {})
                        }
                    )
                    features.append(feature)
    
    feature_collection = geojson.FeatureCollection(features)
    
    # Save to GeoJSON file
    with open(f'{location_name.lower()}_power.geojson', 'w') as geojson_file:
        geojson.dump(feature_collection, geojson_file, indent=2)
    
    print(f"GeoJSON data has been saved to '{location_name.lower()}_power.geojson'")
else:
    print(f"Error: {response.status_code}")       

Successfully downloaded primary data for New Hampshire
Successfully downloaded secondary data for New Hampshire
Successfully downloaded tertiary data for New Hampshire
Successfully downloaded motorway data for New Hampshire
Successfully downloaded trunk data for New Hampshire
Successfully downloaded road data for New Hampshire
Successfully downloaded primary data for New York
Successfully downloaded secondary data for New York
Successfully downloaded tertiary data for New York
Successfully downloaded motorway data for New York
Successfully downloaded trunk data for New York
Successfully downloaded road data for New York
Successfully downloaded primary data for New Mexico
Successfully downloaded secondary data for New Mexico
Successfully downloaded tertiary data for New Mexico
Successfully downloaded motorway data for New Mexico
Successfully downloaded trunk data for New Mexico
Successfully downloaded road data for New Mexico
Successfully downloaded primary data for North Carolina
Succe

In [3]:
import os
root_path = os.path.abspath('../')
raw_data_path = os.path.join(root_path,"raw_data")
file_name= os.path.join(raw_data_path,f"california_roads/motorway_road_location.geojson")
file_name

'/home/kalimdorfolk/code/mashafif/illuminating_horizons/raw_data/california_roads/motorway_road_location.geojson'

In [7]:
import overpass
import geojson

api = overpass.API(timeout=600)
road_type =["primary","secondary","tertiary","motorway","trunk","road"]
root_path = os.path.abspath('../')
raw_data_path = os.path.join(root_path,"raw_data")

for road in road_type:
    try:
        query = f"""
        way["highway"~"^({road})$"](around:5000,35.682839,139.759455);
        out geom;
        """    
        result = api.get(query)
        file_name= os.path.join(raw_data_path,f"tokyo_roads/{road}_road_location.geojson")

        with open(file_name,mode="w") as f:
            geojson.dump(result,f)
        print(f"Successfully downloaded {road} data")
    except Exception as e:
        print(f"Error in downloading {road}: {e}")
    else:
        print(result[0:2])        

Successfully downloaded primary data
[{"geometry": {"coordinates": [[139.760462, 35.666778], [139.760611, 35.666725]], "type": "LineString"}, "id": 4848881, "properties": {"highway": "primary", "lanes": "4", "lit": "yes", "maxspeed": "50", "name": "昭和通り", "name:en": "Showa-dori", "name:ja": "昭和通り", "name:ja_rm": "Shōwa Dōri", "oneway": "yes", "ref": "316", "source": "survey;bing;Bing 2010", "surface": "paved"}, "type": "Feature"}, {"geometry": {"coordinates": [[139.748049, 35.641643], [139.748077, 35.641701], [139.74811, 35.641745], [139.748156, 35.641789], [139.7482, 35.641822], [139.748248, 35.64185], [139.748294, 35.641873], [139.748388, 35.641913], [139.748461, 35.641947], [139.748758, 35.64208], [139.748998, 35.642179], [139.749678, 35.642473], [139.749803, 35.642524], [139.749925, 35.642581], [139.750461, 35.642815], [139.750765, 35.642931], [139.75086, 35.642946], [139.750964, 35.64295], [139.751031, 35.64295]], "type": "LineString"}, "id": 4848884, "properties": {"foot": "no", 

In [37]:
type(result)

geojson.feature.FeatureCollection

In [41]:
with open("./road_location.geojson",mode="w") as f:
  geojson.dump(result,f)

In [29]:
!pwd

/home/kalimdorfolk/code/mashafif/illuminating_horizons/data_exploration


In [ ]:
# Combine results into a single dataset
combined_results = overpy.Result()
for result in results:
    combined_results.nodes.extend(result.nodes)
    combined_results.ways.extend(result.ways)
    combined_results.relations.extend(result.relations)

# The combined_results now contains data from all queries